In [2]:
import pandas as pd
import numpy as np

## Data Preparation

In [3]:
#load data
df_seq=pd.read_csv('pdb_data_seq.csv')
df_properties=pd.read_csv('pdb_data_no_dups.csv')
df_total=df_seq.merge(df_properties,left_on='structureId',right_on = 'structureId')


In [4]:
df_total.columns

Index(['structureId', 'chainId', 'sequence', 'residueCount_x',
       'macromoleculeType_x', 'classification', 'experimentalTechnique',
       'macromoleculeType_y', 'residueCount_y', 'resolution',
       'structureMolecularWeight', 'crystallizationMethod',
       'crystallizationTempK', 'densityMatthews', 'densityPercentSol',
       'pdbxDetails', 'phValue', 'publicationYear'],
      dtype='object')

#### Select only protein, and filtered by top N

In [5]:
#select the data in top n by count, top 10?
count = df_total['classification'].value_counts(dropna=False)[:10]
df_selected=df_total[df_total['classification'].isin(set(count.index))]
#we want only protein
df_selected=df_selected[df_selected['macromoleculeType_x'].isin(set(['Protein']))]

#### Select proteins with only one chain

In [6]:
#select proteins with only one chain in the data set
#how to justify this operation?
#df_onechain = df_selected[df_selected.groupby('structureId').structureId.transform(len) == 1]

In [7]:
test_df = df_selected[['structureId','classification','sequence']]
#test_df = df_onechain[['structureId','classification','sequence']]

Things to be done

Further select data and simplify problem, select proteins with only one chain?


Figure out how to convert sequence data into array and training model afterwards.


More models and discussion (*LSTM)

Models build on features other than sequence.



## Model Training

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier

In [9]:
#optional, take part of the data for faster verification
data = test_df.sample(50000)

#need to remove nulls
data = data.dropna()
X_train, X_test,y_train,y_test = \
train_test_split(data['sequence'], data['classification'], test_size = 0.1, random_state = 1)

#### Feature Extraction From Sequence Data

In [33]:
#vectorize data, prepare for building models
#Convert a collection of text documents to a matrix of token counts
#seems has nothing to do with sequence but only with the frequency

#ngram is a parameter we need to focus on, 

vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (10,10))
#vect = CountVectorizer(analyzer = 'char_wb')
#vect =TfidfVectorizer(analyzer = "char_wb",sublinear_tf= True )
# Fit and Transform CountVectorizer
#occasionally may meet np.nan error
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)

#to store the results for different mothods
prediction = dict()

In [34]:
X_train_df

<45000x3762531 sparse matrix of type '<class 'numpy.int64'>'
	with 12938264 stored elements in Compressed Sparse Row format>

#### Naive Bayes

In [11]:
model = MultinomialNB()
model.fit(X_train_df, y_train)
#test on test set
NB_pred = model.predict(X_test_df)
prediction["MultinomialNB"] = accuracy_score(NB_pred, y_test)
print( prediction['MultinomialNB'])

0.2788


#### adaboost

In [12]:
model = AdaBoostClassifier()
model.fit(X_train_df,y_train)
ADA_pred = model.predict(X_test_df)
prediction["Adaboost"] = accuracy_score(ADA_pred , y_test)
print(prediction["Adaboost"])

0.3102


#### RandomForestClassifier

In [13]:
model = RandomForestClassifier()
model.fit(X_train_df,y_train)
ADA_pred = model.predict(X_test_df)
prediction["Random_Forest"] = accuracy_score(ADA_pred , y_test)
print(prediction["Random_Forest"])

C:\Users\Weinan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8738


#### Neural Network

In [12]:
X_train_df

<45000x26 sparse matrix of type '<class 'numpy.int64'>'
	with 906486 stored elements in Compressed Sparse Row format>

In [13]:
X_test_df

<5000x26 sparse matrix of type '<class 'numpy.int64'>'
	with 101000 stored elements in Compressed Sparse Row format>

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(9736, 2000)
        self.bc1 = nn.BatchNorm1d(2000)
        
        self.fc2 = nn.Linear(2000, 252)
        self.bc2 = nn.BatchNorm1d(252)
        
        self.fc3 = nn.Linear(252, 10)
        
        
    def forward(self, x):
        x = x.view((-1, 9736))
        h = self.fc1(x)
        h = self.bc1(h)
        h = F.relu(h)
        #h = F.dropout(h, p=0.5, training=self.training)
        
        h = self.fc2(h)
        h = self.bc2(h)
        h = F.relu(h)
        #h = F.dropout(h, p=0.2, training=self.training)
        
        h = self.fc3(h)
        out = F.log_softmax(h,dim = 1)
        return out

In [ ]:
model = Model()
model.cuda() # CUDA!
optimizer = optim.Adam(model.parameters(), lr=0.005)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Prepare data for pytorch

In [ ]:
#convert y labels to index encoding
y_train_array = np.zeros(162111)
for i in range(len(y_train)):
    cur_key = y_train.iloc[i]
    idx = cls.index(cur_key)
    y_train_array[i]=idx

In [ ]:
y_test_array = np.zeros(18013)
for i in range(len(y_test)):
    cur_key = y_test.iloc[i]
    idx = cls.index(cur_key)
    y_test_array[i]=idx

In [ ]:
"""
criterion = nn.CrossEntropyLoss()
losses = []
for epoch in range(5):
    
    for i in range(1600):
        data = torch.from_numpy(X_train_df[i:i+100].toarray()).float().cuda()
        target = torch.from_numpy(y_train_array[i:i+100]).to(device = device, dtype=torch.int64)
        #print(data)
        # Get Samples
        data, target = Variable(data.cuda()), Variable(target.cuda())
        #target = target.squeeze(1)
        # Init
        optimizer.zero_grad()

        # Predict
        #print(type(data))
        y_pred = model(data) 
        #print(y_pred)
        # Calculate loss
        loss = criterion(y_pred, target)
        losses.append(loss.item())
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        # Display
        if i % 100 == 1:
            print(loss.item())
            
    print()
    
    
    #test on test set
    test_acc = 0
    for i in range(180):
        data = torch.from_numpy(X_test_df[i:i+100].toarray()).float().cuda()
        target = torch.from_numpy(y_test_array[i:i+100]).to(device = device, dtype=torch.int64)
        
        y_pred = model(data)
        _,predicted = torch.max(y_pred,1)
        test_acc += (predicted == target).sum().item()
        
        #print('temp accuracy ',(predicted == target).sum().item()/100)
        
    print('current test_accuracy',test_acc/18000)
"""

Another vectoriation method

### RNN

In [15]:
#https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import torch
all_letters = string.ascii_letters[26:]
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
def findFiles(path): return glob.glob(path)

In [16]:
all_letters

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [17]:
category_lines = {}
all_categories = []

In [18]:

all_categories= list(count.index)

### Turning seq to tensors

In [19]:
for i in range(len(X_train)):
    if y_train.iloc[i] not in category_lines:
        category_lines[y_train.iloc[i]] = [X_train.iloc[i]]
    else:
        category_lines[y_train.iloc[i]].append(X_train.iloc[i])
n_categories = len(category_lines)

In [20]:
def letterToIndex(letter):
    return all_letters.find(letter)
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [88]:
X_train_tensor = lineToTensor(X_train.values)

In [90]:
X_train_tensor.size()

torch.Size([45000, 1, 26])

In [65]:
import torch.nn as nn
import torch.nn.functional as F
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        #self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers)
        
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM,self).__init__()
        
        
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.lstm2out = nn.Linear(hidden_size,output_size)

    def forward(self, seq):
        output,_ = self.lstm(seq.view(len(seq), 1, -1))

        #output,self.hidden = self.lstm(output.view(len(seq),1,-1),self.hidden)
        
        output = self.lstm2out(output)[0]
        #print(output)
        
        return F.log_softmax(output, dim=1)
n_hidden = 128


In [66]:
#rnn = RNN(n_letters, n_hidden, n_categories)#.to('cuda:0')
rnn = LSTM(n_letters, n_hidden, n_categories)

In [39]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i


In [40]:
import random
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)#.to('cuda:0')
    line_tensor = lineToTensor(line)#.to('cuda:0')
    return category, line, category_tensor, line_tensor

category, line, category_tensor, line_tensor = randomTrainingExample()
print('category =', category, '/ sequence =', line)


category = LYASE / sequence = SDSLSKSPENWMSKLDDGKHLTEINIPGSHDSGSFTLKDPVKSVWAKTQDKDYLTQMKSGVRFFDIRGRASADNMISVHHGMVYLHHELGKFLDDAKYYLSAYPNETIVMSMKKDYDSDSKVTKTFEEIFREYYYNNPQYQNLFYTGSNANPTLKETKGKIVLFNRMGGTYIKSGYGADTSGIQWADNATFETKINNGSLNLKVQDEYKDYYDKKVEAVKNLLAKAKTDSNKDNVYVNFLSVASGGSAFNSTYNYASHINPEIAKTIKANGKARTGWLIVDYAGYTWPGYDDIVSEIIDSNKLEHHHHHH


In [67]:
from tqdm import tqdm
import time
import math
criterion = nn.CrossEntropyLoss()
n_iters = 2000
print_every = 50
plot_every = 1000


# Keep track of losses for plotting
current_loss = 0
all_losses = []

learning_rate = 0.01 # If you set this too high, it might explode. If too low, it might not learn
#import progressbar
#progress = progressbar.ProgressBar()
def train(category_tensor, line_tensor):
    #hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        #print(line_tensor[i])
        #utput, hidden = rnn(line_tensor[i], hidden)
        output = rnn(line_tensor[i])
    loss = criterion(output, category_tensor)
    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in tqdm(range(1, n_iters + 1)):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    
    
    
    
    category_tensor = category_tensor
    line_tensor = line_tensor
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0





  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]



  0%|                                                                                 | 1/2000 [00:00<05:13,  6.37it/s]



  0%|                                                                                 | 2/2000 [00:00<05:40,  5.87it/s]



  0%|                                                                                 | 3/2000 [00:00<06:04,  5.48it/s]



  0%|▏                                                                                | 4/2000 [00:00<05:18,  6.26it/s]



  0%|▏                                                                                | 5/2000 [00:00<04:49,  6.90it/s]



  0%|▏                                                                                | 6/2000 [00:00<05:26,  6.10it/s]



  0%|▎                                                                                | 7/2000 [00:01<06:39,  4.99it/s]



  0%|▎      

50 2% (0m 8s) 2.3390 PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEGATPQDLNTMLNTVGGHQAAMQMLKETINEEAAEWDRLHPVHAGPIEPGQMREPRGSDIAGTTSTLQEQIGWMTHNPPIPVGEIYKRWIILGLNKIVRMYSPTSILDIRQGPKEPFRDYVDRFYKTLRAEQASQEVKNWMTETLLVQNANPDCKTILKALGPAATLEEMMTACQGVGGPGHKARVL / TRANSFERASE ✗ (VIRUS)






  2%|██                                                                              | 50/2000 [00:08<05:44,  5.67it/s]



  3%|██                                                                              | 51/2000 [00:08<05:22,  6.04it/s]



  3%|██                                                                              | 53/2000 [00:08<05:02,  6.44it/s]



  3%|██▏                                                                             | 54/2000 [00:08<05:00,  6.47it/s]



  3%|██▏                                                                             | 55/2000 [00:08<04:29,  7.21it/s]



  3%|██▏                                                                             | 56/2000 [00:09<04:56,  6.55it/s]



  3%|██▎                                                                             | 57/2000 [00:09<04:48,  6.73it/s]



  3%|██▎                                                                             | 58/2000 [00:09<05:33,  5.82it/s]



  3%|██▍    

100 5% (0m 14s) 2.1899 GDMSHKEFTKFCYEVYNEIKISDKEFKEKRAALDTLRLCLKRISPDAELVAFGSLESGLALKNSDMDLCVLMDSRVQSDTIALQFYEELIAEGFEGKFLQRARIPIIKLTSDTKNGFGASFQCAIGFNNRLAIHNTLLLSSYTKLDARLKPMVLLVKHWAKRKQINSPYFGTLSSYGYVLMVLYYLIHVIKPPVFPNLLLSPLKQEKIVDGFDVGFDDKLEDIPPSQNYSSLGSLLHGFFRFYAYKFEPREKVVTFRRPDGYLTKQEKGWTSATEHTGSADQIIKDRYILAIEDPFEISHNVGRTVSSSGLYRIRGEFMAASRLLNSRSYPIPYDSLFEEA / TRANSFERASE ✓






  5%|███▉                                                                           | 100/2000 [00:14<04:30,  7.02it/s]



  5%|████                                                                           | 102/2000 [00:14<03:55,  8.07it/s]



  5%|████                                                                           | 103/2000 [00:15<04:32,  6.96it/s]



  5%|████                                                                           | 104/2000 [00:15<07:10,  4.40it/s]



  5%|████▏                                                                          | 106/2000 [00:15<06:44,  4.68it/s]



  5%|████▏                                                                          | 107/2000 [00:16<06:02,  5.22it/s]



  5%|████▎                                                                          | 108/2000 [00:16<05:20,  5.91it/s]



  5%|████▎                                                                          | 109/2000 [00:16<05:11,  6.07it/s]



  6%|████▎  

...





 95%|██████████████████████████████████████████████████████████████████████████    | 1900/2000 [04:47<00:14,  6.75it/s]



 95%|██████████████████████████████████████████████████████████████████████████▏   | 1903/2000 [04:48<00:13,  7.26it/s]



 95%|██████████████████████████████████████████████████████████████████████████▎   | 1905/2000 [04:48<00:11,  8.36it/s]



 95%|██████████████████████████████████████████████████████████████████████████▎   | 1906/2000 [04:48<00:11,  8.21it/s]



 95%|██████████████████████████████████████████████████████████████████████████▍   | 1908/2000 [04:48<00:10,  8.87it/s]



 95%|██████████████████████████████████████████████████████████████████████████▍   | 1909/2000 [04:49<00:13,  6.68it/s]



 96%|██████████████████████████████████████████████████████████████████████████▍   | 1910/2000 [04:49<00:15,  5.77it/s]



 96%|██████████████████████████████████████████████████████████████████████████▌   | 1911/2000 [04:49<00:15,  5.68it/s]



 96%|███████

1950 97% (4m 56s) 2.2738 MKLAPYILELLTSVNRTNGTADLLVPLLRELAKGRPVSRTTLAGILDWPAERVAAVLEQATSTEYDKDGNIIGYGLTLRETSYVFEIDDRRLYAWCALDTLIFPALIGRTARVSSHCAATGAPVSLTVSPSEIQAVEPAGMAVSLVLPQEAADVRQSFCCHVHFFASVPTAEDWASKHQGLEGLAIVSVHEAFGLGQEFNRHLLQTMSSRTPLEHHHHHH / TRANSCRIPTION ✗ (LYASE)






 98%|████████████████████████████████████████████████████████████████████████████  | 1950/2000 [04:56<00:09,  5.21it/s]



 98%|████████████████████████████████████████████████████████████████████████████  | 1951/2000 [04:56<00:10,  4.63it/s]



 98%|████████████████████████████████████████████████████████████████████████████▏ | 1952/2000 [04:56<00:09,  5.15it/s]



 98%|████████████████████████████████████████████████████████████████████████████▏ | 1953/2000 [04:56<00:09,  4.84it/s]



 98%|████████████████████████████████████████████████████████████████████████████▏ | 1954/2000 [04:57<00:08,  5.36it/s]



 98%|████████████████████████████████████████████████████████████████████████████▏ | 1955/2000 [04:57<00:08,  5.61it/s]



 98%|████████████████████████████████████████████████████████████████████████████▎ | 1956/2000 [04:57<00:07,  5.90it/s]



 98%|████████████████████████████████████████████████████████████████████████████▎ | 1958/2000 [04:57<00:06,  6.90it/s]



 98%|███████

2000 100% (5m 5s) 2.4378 MHHHHHHSSGVDLGTENLYFQSMPAPYWEGTAVIDGEFKELKLTDYRGKYLVFFFYPLDFTFVCPTEIIAFGDRLEEFRSINTEVVACSVDSQFTHLAWINTPRRQGGLGPIRIPLLSDLTHQISKDYGVYLEDSGHTLRGLFIIDDKGILRQITLNDLPVGRSVDETLRLVQAFQYTDKHGEVCPAGWKPGSETIIPDPAGKLKYFDKLN / TRANSCRIPTION ✗ (OXIDOREDUCTASE)






100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [05:05<00:00,  5.35it/s]





In [401]:
guess, guess_i = categoryFromOutput(output)

In [402]:
guess

'RIBOSOME'

In [403]:
guess_i

0

In [399]:
category_i

NameError: name 'category_i' is not defined

In [391]:
import progressbar

LSTM

In [151]:
class Params():
    def __init__(self):
        self.batch_size = 32
        self.hidden_size = 12
        self.embedding_size = 24
        self.epochs = 50
        self.nr_classes =2
        self.gpu = False
        self.learning_rate = 0.001
        self.train_ratio = 0.82
        self.val_ratio = 0.1
        self.test_ratio = 1-self.val_ratio-self.train_ratio
        self.n_layers = 2
        self.dropout = 0.3


Args = Params()

In [74]:
import torch.autograd as autograd
class LSTMClassifier(nn.Module):

    def __init__(self, input_size, hidden_dim, label_size, batch_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        #self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, sentence):
        #embeds = self.word_embeddings(sentence)
        x = sentence.view(len(sentence), self.batch_size , -1)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y  = self.hidden2label(lstm_out[-1])
        log_probs = F.log_softmax(y)
        return log_probs
    
    
def train_epoch(model, X_train, y_train,loss_function,optimizer):
    model.train()
    avg_loss = 0.0
    count = 0
    truth_res = []
    pred_res = []
    for i in range(1000):
        #sent, label = batch.text, batch.label
        #label.data.sub_(1)
        #truth_res += list(label.data)
        model.batch_size = 10
        label = y_train[i:i+model.batch_size]
        model.hidden = model.init_hidden()# detaching it from its history on the last instance.
        pred = model(X_train[i:i+model.batch_size])
        pred_label = pred.data.max(1)[1].numpy()
        pred_res += [x[0] for x in pred_label]
        model.zero_grad()
        loss = loss_function(pred, label)
        avg_loss += loss.data[0]
        count += 1
        if i % 10 == 0:
            print('epoch: %d iterations: %d loss :%g' % (i, count*model.batch_size, loss.data[0]))
        loss.backward()
        optimizer.step()
    #avg_loss /= len(train_iter)
    #print('epoch: %d done!\ntrain avg_loss:%g , acc:%g'%(i, avg_loss, get_accuracy(truth_res,pred_res)))


In [92]:
count

RIBOSOME                         60710
HYDROLASE                        47833
TRANSFERASE                      37726
OXIDOREDUCTASE                   35114
IMMUNE SYSTEM                    15989
LYASE                            11871
HYDROLASE/HYDROLASE INHIBITOR    11262
VIRUS                            10832
TRANSCRIPTION                    10564
VIRAL PROTEIN                     8875
Name: classification, dtype: int64

In [75]:
model = LSTMClassifier(n_letters,128,10,10)
loss_function =  nn.CrossEntropyLoss()
optimizer =  optim.Adam(model.parameters(), lr=0.005)


In [189]:
def train(input_tensor, input_sizes, target_tensor):
    model.hidden = model.init_hidden(Args.batch_size) 
    model.zero_grad()
    output, hidden = model.forward(input_tensor, input_sizes)
    loss = criterion(output, target_tensor)
    loss.backward()
    optimizer.step()
    return output, loss.item()

In [192]:
model = LSTM_classifier(vocab_size = n_letters, 
                        hidden_size = Args.hidden_size, 
                        n_layers = Args.n_layers, 
                        dropout = Args.dropout,
                        embed_size = Args.embedding_size).cuda()
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [191]:
n_letters

26

In [179]:
test = y_train.iloc[list(range(0,10))]

In [181]:
test.values

array(['HYDROLASE', 'TRANSFERASE', 'HYDROLASE', 'OXIDOREDUCTASE',
       'HYDROLASE', 'HYDROLASE', 'LYASE', 'LYASE', 'HYDROLASE',
       'TRANSFERASE'], dtype=object)

In [201]:
label_map['TRANSFERASE']

2

In [218]:
import time
from torch.autograd import Variable

current_loss = 0

all_losses = []
val_acc = []
train_acc = []
predictions = []
labels = []
#print_every = int(len(train_data)/Args.batch_size) #every epoch
#n_iters  = int(len(train_data)/Args.batch_size*Args.epochs)
#plot_every = print_every
start = time.time()
epochs = 10
batch_size = 32
label_map = dict(zip(list(count.index),list(range(10))))
for i in range(epochs):
    for idx in range(len(X_train)%batch_size):
        cur_x = X_train.iloc[list(range(idx,idx+batch_size))].values
        temp_y = y_train.iloc[list(range(idx,idx+batch_size))].values
        #print(type(cur_y))
        #print(cur_y[0])
        cur_x = lineToTensor(cur_x)
        cur_y = np.zeros(batch_size)
        for i in range(len(cur_y)):
            cur_y[i] = label_map[temp_y[i]]
        #print(type(cur_y))
        cur_y = torch.from_numpy(cur_y.astype(np.int))
        input_lengths = [len(x) for x in cur_x]
        print(cur_y)
        #print(cur_x)
        print(input_lengths)
        output, loss = train(input_tensor=cur_x, 
                     input_sizes=input_lengths, 
                     target_tensor=cur_y)
        current_loss += loss
        predictions.extend(list(torch.argmax(output, dim=1).numpy()))
        #labels.extend(list(target_tensor.numpy()))

                
                
        
        
        
        
        

tensor([1, 2, 1, 3, 1, 1, 5, 5, 1, 2, 1, 1, 1, 1, 4, 1, 2, 2, 5, 2, 1, 1, 1, 2,
        2, 4, 4, 9, 5, 1, 1, 1], dtype=torch.int32)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got CPUType instead (while checking arguments for embedding)

What about top-5 accuracy

And you need to balance the weight for each category